在labelme標記完之後，原圖放在images資料夾裡、json放在labelme_jsons資料夾裡
接下來要產生的mask放到annotations裡面

In [1]:
import os
import json
from PIL import Image, ImageDraw
import tkinter as tk
from tkinter import filedialog
import shutil
import random

def create_mask(image_path, json_path, output_folder):
    # 讀取json檔案
    with open(json_path, 'r', encoding='utf-8') as f:
        data = json.load(f)

    # 取得原始圖片的大小
    image = Image.open(image_path)
    width, height = image.size

    # 創建一個黑色底的遮罩
    mask = Image.new('L', (width, height), 0)
    draw = ImageDraw.Draw(mask)

    # 繪製每個splash的區域
    for shape in data['shapes']:
        label = shape['label']
        points = shape['points']
        # 將每個splash的座標轉換成整數
        points = [(int(x), int(y)) for x, y in points]
        draw.polygon(points, fill=255)

    # 儲存遮罩圖片
    mask_path = os.path.join(output_folder, os.path.basename(image_path).replace('.png', '_mask.png'))
    mask.save(mask_path)

def process_folders(images_folder, labelme_jsons_folder, output_folder):
    # 創建輸出遮罩的資料夾
    os.makedirs(output_folder, exist_ok=True)

    # 處理每張圖片和對應的json檔案
    for filename in os.listdir(images_folder):
        if filename.endswith('.png'):
            image_path = os.path.join(images_folder, filename)
            json_path = os.path.join(labelme_jsons_folder, filename.replace('.png', '.json'))
            create_mask(image_path, json_path, output_folder)

    print("遮罩生成完成。")


def split_png_dataset(root_folder, split_ratio=0.8):
    # 創建 training 和 validation 子資料夾
    os.makedirs(os.path.join(root_folder, 'training'), exist_ok=True)
    os.makedirs(os.path.join(root_folder, 'validation'), exist_ok=True)

    # 取得所有 PNG 檔案
    png_files = [f for f in os.listdir(root_folder) if f.endswith('.png')]

    # 計算用於訓練的檔案數量
    num_train = int(len(png_files) * split_ratio)

    # 隨機選擇用於訓練的檔案
    random.seed(42)
    train_files = random.sample(png_files, num_train)

    # 將選擇的檔案搬移到 training 子資料夾
    for file in train_files:
        shutil.move(os.path.join(root_folder, file), os.path.join(root_folder, 'training', file))
    
    # 將剩餘的檔案搬移到 validation 子資料夾
    for file in png_files:
        if file not in train_files:
            shutil.move(os.path.join(root_folder, file), os.path.join(root_folder, 'validation', file))
    
    print("資料集拆分完成。")

def select_parent_folder():
    parent_folder = filedialog.askdirectory(title="選擇資料夾A")
    return parent_folder

# 使用tkinter來選擇資料夾A
root = tk.Tk()
root.withdraw()  # 隱藏主視窗

parent_folder = select_parent_folder()
if parent_folder:
    images_folder = os.path.join(parent_folder, 'images')
    labelme_jsons_folder = os.path.join(parent_folder, 'labelme_jsons')
    annotations_folder = os.path.join(parent_folder, 'annotations')

    process_folders(images_folder, labelme_jsons_folder, annotations_folder)
    print("---------")

    train_ratio = float(input("請輸入訓練集比例（0到1之間，例如0.8）："))

    # 執行資料集拆分
    split_png_dataset(images_folder, train_ratio)
    split_png_dataset(annotations_folder, train_ratio)


遮罩生成完成。
---------
資料集拆分完成。
資料集拆分完成。
